In [12]:
import numpy as np
# constant size
size = 1000
# Set p(H) apriori for simulation as a biased coin.
p_h = 0.5

# Model experiment as a single biased coin flipped 1000 times.
data = np.random.binomial(1, p_h, size=size)

In [13]:
n_states = 2
n_obs = 2

emission = np.random.rand(size,n_states)
emission = emission/np.tile(np.expand_dims(np.sum(emission,axis=1),axis=1),(1,2))

transition = np.array([[0.1,0.9],[0.9,0.1]])


forward = np.random.rand(size,n_states)
scale_factors = np.zeros((size))
forward_hat = np.zeros((size,n_states))

backward = np.random.rand(size,n_states)
backward_hat = np.zeros((size,n_states))

init_prob = np.array([0.5,0.5])
init_obs = np.zeros(n_states)

p_old = -10000
tol = 0.001
max_iter = 100

mu = np.random.rand(n_states)

In [14]:
for ite in range(max_iter):
    forward[0,:] = init_prob * emission[0,:]
    scale_factors[0] = np.sum(forward[0,:])
    forward_hat[0,:] = forward[0,:]/scale_factors[0]
    
    for t in range(size-1):
        temp = np.matmul(forward_hat[t,:] ,transition) * emission[t+1,:]
        scale_factors[t+1] = np.sum(temp)
        forward_hat[t+1,:] = temp/scale_factors[t+1]
        forward[t+1,:] = forward_hat[t+1,:]*np.prod(scale_factors[0:t+1])

    backward[-1,:] = 1
    backward_hat[-1,:] = backward[-1,:]
    for t in reversed(range(size-1)):
        temp = np.matmul(backward_hat[t+1,:]*emission[t+1,:],np.transpose(transition))
        backward_hat[t,:] = temp/scale_factors[t+1]
        backward[t,:] = backward_hat[t,:]*np.prod(scale_factors[t+1:-1])


    a = np.zeros((size,n_states))
    b = np.zeros((size,n_states,n_states))
    for i in range(size):
        for j in range(n_states):
            a[i,j] = forward_hat[i,j]*backward_hat[i,j]
    for t in range(size-1):
        for i in range(n_states):
            for j in range(n_states):
                b[t,i,j] = scale_factors[t+1]*forward_hat[t,i]*backward_hat[t+1,j]*transition[i,j]*emission[j,data[t+1]]

    for i in range(n_states):
        for j in range(n_states):
            transition[i,j] = np.sum(b[0:-1,i,j])/np.sum(b[0:-1,i,:])

    for i in range(n_states):
        init_prob[i] = a[0,i]/np.sum(a[0,:])


    for j in range(n_states):
        emission[:,j] = np.sum(a[np.argwhere(data==j),j]) / np.sum(a[:,j])
            
    p = np.sum(np.log(scale_factors))
    print(f'p is:{p}')
    # print(f'transition prob is: {transition}')
    # print(f'emission prob is:{emission}')
    # if p - p_old < tol:
    #     break
    p_old = p

p is:-723.0570062773961
p is:-682.2610851822665
p is:-693.55065126645
p is:-693.6104625215752
p is:-693.6703156803047
p is:-693.7300559733178
p is:-693.7896790503091
p is:-693.8491806881063
p is:-693.9085567938562
p is:-693.9678034163218
p is:-694.0269167567895
p is:-694.0858931795145
p is:-694.1447292216562
p is:-694.2034216026457
p is:-694.2619672329427
p is:-694.3203632221264
p is:-694.3786068862917
p is:-694.4366957546965
p is:-694.4946275756465
p is:-694.5524003215755
p is:-694.6100121933026
p is:-694.6674616234537
p is:-694.7247472790236
p is:-694.7818680630852
p is:-694.8388231156318
p is:-694.8956118135636
p is:-694.95223376982
p is:-695.0086888316775
p is:-695.0649770782261
p is:-695.1210988170467
p is:-695.1770545801207
p is:-695.232845118996
p is:-695.2884713992485
p is:-695.3439345942767
p is:-695.3992360784662
p is:-695.4543774197723
p is:-695.509360371763
p is:-695.5641868651765
p is:-695.6188589990318
p is:-695.673379031352
p is:-695.7277493695501
p is:-695.7819725605268

In [15]:
transition

array([[0.35334018, 0.64665982],
       [0.96868785, 0.03131215]])

In [16]:
init_prob

array([0.08398874, 0.91601126])

In [17]:
data[0]

0